In [1]:
from pathlib import Path
import shutil

# Your downloaded folder structure
source_dir = Path('project-2-at-2026-01-20-08-57-04ab19ec')  # CHANGE THIS

# Target structure
target_dir = Path('yolo_obb_training')

# Create directories
(target_dir / 'images' / 'train').mkdir(parents=True, exist_ok=True)
(target_dir / 'images' / 'val').mkdir(parents=True, exist_ok=True)
(target_dir / 'labels' / 'train').mkdir(parents=True, exist_ok=True)
(target_dir / 'labels' / 'val').mkdir(parents=True, exist_ok=True)

# Copy files
shutil.copytree(source_dir / 'images', target_dir / 'images' / 'train', dirs_exist_ok=True)
shutil.copytree(source_dir / 'labels', target_dir / 'labels' / 'train', dirs_exist_ok=True)
shutil.copy(source_dir / 'classes.txt', target_dir / 'classes.txt')

print("✅ Files organized")

✅ Files organized


In [2]:
from sklearn.model_selection import train_test_split
import shutil
from pathlib import Path

train_dir = Path('yolo_obb_training')

# Get all images
images = list((train_dir / 'images' / 'train').glob('*.jpg'))
print(f"Total images: {len(images)}")

# Split 80/20
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

# Move validation files
for img in val_imgs:
    label = train_dir / 'labels' / 'train' / f'{img.stem}.txt'
    
    # Move image
    shutil.move(str(img), str(train_dir / 'images' / 'val' / img.name))
    
    # Move label
    if label.exists():
        shutil.move(str(label), str(train_dir / 'labels' / 'val' / f'{img.stem}.txt'))

print(f"✅ Train: {len(train_imgs)}, Val: {len(val_imgs)}")

Total images: 52
✅ Train: 41, Val: 11


In [4]:
from ultralytics import YOLO

# Load OBB model
model = YOLO('yolov8n-obb.pt')

# Train
results = model.train(
    data='mandible_obb.yaml',
    epochs=150,      # Ditambah karena data sedikit butuh waktu konvergensi lebih
    imgsz=640,
    batch=8,         # batch 16 mungkin terlalu berat untuk stabilitas gradient pada N=52
    patience=30,
    
    # OBB & Medical Augmentation
    degrees=10.0,
    translate=0.15,
    scale=0.4,
    fliplr=0.5,
    mosaic=0.0,      # KRUSIAL: Matikan untuk data medis
    
    # Akselerasi M4 Mac
    device='mps',    
    project='mandible_yolo_obb',
    name='exp_v1_n52'
)

Ultralytics 8.4.6 🚀 Python-3.10.19 torch-2.9.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=mandible_obb.yaml, degrees=10.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-obb.pt, momentum=0.937, mosaic=0.0, multi_scale=0.0, name=exp_v1_n52, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, perspective=0.0, plots=True, pose=12.0, pretra

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd

class OBBMandibleCropper:
    def __init__(self, model_path, margin_percent=0.1):
        self.model = YOLO(model_path)
        self.margin_percent = margin_percent  # 10% margin
        
    def add_margin(self, points, margin_percent=0.1):
        """Add safety margin to OBB points"""
        center = points.mean(axis=0)
        expanded = center + (points - center) * (1 + margin_percent)
        return expanded
    
    def make_square_with_padding(self, image):
        """Add black padding to make image square (preserve aspect ratio)"""
        h, w = image.shape[:2]
        
        if h == w:
            return image
        
        # Determine size of square
        size = max(h, w)
        
        # Create black canvas
        square = np.zeros((size, size), dtype=image.dtype)
        
        # Calculate padding
        y_offset = (size - h) // 2
        x_offset = (size - w) // 2
        
        # Place image in center
        square[y_offset:y_offset+h, x_offset:x_offset+w] = image
        
        return square
    
    def process_image(self, img_path, save_dir):
        img = c
        
        
        .imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            return {'status': 'failed', 'reason': 'cannot_read'}
        
        results = self.model.predict(img_path, conf=0.5, verbose=False)
        
        metadata = {
            'filename': Path(img_path).name,
            'status': 'success',
            'detections': []
        }
        
        for result in results:
            if result.obb is None:
                metadata['status'] = 'no_detection'
                continue
                
            for obb in result.obb:
                points = obb.xyxyxyxy[0].cpu().numpy()
                cls = int(obb.cls[0])
                conf = float(obb.conf[0])
                class_name = self.model.names[cls]
                
                # Add margin
                points_expanded = self.add_margin(points, self.margin_percent)
                
                # Get OBB rectangle
                rect = cv2.minAreaRect(points_expanded.astype(np.float32))
                center, (width, height), angle = rect
                
                # ============================================
                # FIX: Correct orientation
                # ============================================
                
                # Ensure width > height (horizontal orientation)
                if height > width:
                    width, height = height, width
                    angle += 90
                
                # Get rotation matrix
                M_rotate = cv2.getRotationMatrix2D(center, angle, 1.0)
                
                # Rotate entire image
                img_h, img_w = img.shape[:2]
                rotated = cv2.warpAffine(img, M_rotate, (img_w, img_h), 
                                        flags=cv2.INTER_CUBIC,
                                        borderMode=cv2.BORDER_REPLICATE)
                
                # Get rotated center
                center_rotated = np.dot(M_rotate, [center[0], center[1], 1])
                
                # Crop aligned rectangle
                x = int(center_rotated[0] - width/2)
                y = int(center_rotated[1] - height/2)
                w = int(width)
                h = int(height)
                
                # Ensure crop is within bounds
                x = max(0, x)
                y = max(0, y)
                w = min(w, img_w - x)
                h = min(h, img_h - y)
                
                cropped = rotated[y:y+h, x:x+w]
                
                # ============================================
                # Check if upside down (optional heuristic)
                # ============================================
                # If bottom half is brighter than top half, flip
                top_half = cropped[:h//2, :]
                bottom_half = cropped[h//2:, :]
                
                if np.mean(bottom_half) > np.mean(top_half) * 1.2:
                    cropped = cv2.rotate(cropped, cv2.ROTATE_180)
                
                # Add padding to make square
                squared = self.make_square_with_padding(cropped)
                
                # Mirror right side
                if class_name == 'mandible_right':
                    squared = cv2.flip(squared, 1)
                
                # Resize
                resized = cv2.resize(squared, (224, 224), 
                                    interpolation=cv2.INTER_CUBIC)
                
                # CLAHE
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
                enhanced = clahe.apply(resized)
                
                # Save
                output_filename = f"{Path(img_path).stem}_{class_name}.png"
                output_path = Path(save_dir) / output_filename
                cv2.imwrite(str(output_path), enhanced)
                
                metadata['detections'].append({
                    'class': class_name,
                    'confidence': conf,
                    'output_file': output_filename
                })
        
        return metadata

    def batch_process(self, input_dir, output_dir):
        """Process all images"""
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        image_files = list(Path(input_dir).glob('*.jpg')) + \
                     list(Path(input_dir).glob('*.png'))
        
        print(f"🔍 Found {len(image_files)} images")
        
        results_log = []
        failed = []
        
        for img_path in tqdm(image_files, desc="Processing OBB"):
            metadata = self.process_image(img_path, output_dir)
            results_log.append(metadata)
            
            if metadata['status'] != 'success':
                failed.append(metadata)
        
        success = sum(1 for r in results_log if r['status'] == 'success')
        total_crops = sum(len(r['detections']) for r in results_log)
        
        print("\n" + "="*60)
        print("📊 OBB PROCESSING SUMMARY")
        print("="*60)
        print(f"✅ Processed: {success}/{len(image_files)}")
        print(f"🖼️  Total crops: {total_crops}")
        print(f"❌ Failed: {len(failed)}")
        
        if failed:
            print("\n⚠️  Failed images:")
            for f in failed[:10]:
                print(f"   - {f['filename']}")
        
        # Save log
        pd.DataFrame(results_log).to_csv(
            Path(output_dir) / 'obb_processing_log.csv', index=False
        )
        
        return results_log

if __name__ == "__main__":
    # Configuration
    MODEL_PATH = "mandible_yolo_obb/exp_v1_n52/weights/best.pt"
    INPUT_DIR = "/Users/rieno/Downloads/DATASET MENTAH/MATCHED_DATASET_3/uncropped"
    OUTPUT_DIR = "cropped_obb_hemijaws"
    
    # Process with 10% safety margin
    cropper = OBBMandibleCropper(MODEL_PATH, margin_percent=0.10)
    results = cropper.batch_process(INPUT_DIR, OUTPUT_DIR)
    
    print(f"\n✅ Done! Check: {OUTPUT_DIR}/")

🔍 Found 483 images


Processing OBB: 100%|██████████| 483/483 [00:29<00:00, 16.18it/s]


📊 OBB PROCESSING SUMMARY
✅ Processed: 483/483
🖼️  Total crops: 967
❌ Failed: 0

✅ Done! Check: cropped_obb_hemijaws/


In [ ]:
from pathlib import Path

# Get all images and labels
images = {img.stem for img in Path('yolo_training/images/train').glob('*.jpg')}
labels = {lbl.stem for lbl in Path('yolo_training/labels/train').glob('*.txt')}

# Find mismatches
images_no_label = images - labels
labels_no_image = labels - images

print("Images without labels:")
for img in sorted(images_no_label):
    print(f"  {img}")
    
    # Try to find similar label names
    for lbl in labels:
        if img[:10] in lbl or lbl[:10] in img:
            print(f"    → Possible match: {lbl}")

print("\nLabels without images:")
for lbl in sorted(labels_no_image):
    print(f"  {lbl}")

Images without labels:
  10_03_AN_NISA_IZZATI_19122018
    → Possible match: 89d3a546-10_03_AN_NISA_IZZATI_19122018
  10_05_RAFI_ANANTA_20122018
    → Possible match: 1d5535c0-10_05_RAFI_ANANTA_20122018
  10_49_An_Primakhansas_Laurelia_F_12052018
    → Possible match: c8a1a7ea-10_49_An_Primakhansas_Laurelia_F_12052018
  10_89_MICHAEL_C_SEWOW_F_21022020
    → Possible match: 4ddf59bb-10_89_MICHAEL_C_SEWOW_F_21022020
  11_95_KHADIJAH_AL_KUBRO_08122021
    → Possible match: 1aecc85c-11_95_KHADIJAH_AL_KUBRO_08122021
  12_02_RAFI_AZZAM_25062019
    → Possible match: 3cddb946-12_02_RAFI_AZZAM_25062019
  13_16_ATILAH_MARCELINO_28042017
    → Possible match: 09c63026-13_16_ATILAH_MARCELINO_28042017
  13_28_MAULIDYA_SABRINA_PR_17072019
    → Possible match: 9a471d67-13_28_MAULIDYA_SABRINA_PR_17072019
  13_78_ALTHAFIYAH_PUSPITA_B_AN_21032018
    → Possible match: 64cb746e-13_78_ALTHAFIYAH_PUSPITA_B_AN_21032018
  14_67_Nuraini_23052016
    → Possible match: faeaa87d-14_67_Nuraini_23052016
  14_82

In [9]:
import cv2
import matplotlib.pyplot as plt
import random
from pathlib import Path
import numpy as np

def verify_obb_crops(crop_dir, n_samples=20):
    """Visual quality check for OBB crops"""
    
    crop_files = list(Path(crop_dir).glob('*.png'))
    samples = random.sample(crop_files, min(n_samples, len(crop_files)))
    
    fig, axes = plt.subplots(4, 5, figsize=(20, 16))
    axes = axes.ravel()
    
    aspect_ratios = []
    
    for idx, img_path in enumerate(samples):
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        
        # Check for distortion
        h, w = img.shape
        aspect_ratio = w / h
        aspect_ratios.append(aspect_ratio)
        
        axes[idx].imshow(img, cmap='gray')
        axes[idx].set_title(f"{img_path.stem[:30]}\nRatio: {aspect_ratio:.2f}", 
                           fontsize=8)
        axes[idx].axis('off')
        
        # Draw center lines to check alignment
        axes[idx].axhline(y=h//2, color='r', linewidth=0.5, alpha=0.3)
        axes[idx].axvline(x=w//2, color='r', linewidth=0.5, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('obb_quality_check.png', dpi=150, bbox_inches='tight')
    print("✅ Saved obb_quality_check.png")
    
    # Statistics
    print("\n" + "="*60)
    print("📊 CROP QUALITY STATISTICS")
    print("="*60)
    print(f"Total crops: {len(crop_files)}")
    print(f"Aspect ratios: {np.mean(aspect_ratios):.3f} ± {np.std(aspect_ratios):.3f}")
    
    if np.std(aspect_ratios) < 0.1:
        print("✅ Excellent consistency - all crops are square")
    else:
        print("⚠️  Aspect ratio variance detected - check padding logic")

# Run verification
verify_obb_crops('cropped_obb_hemijaws', n_samples=20)

/var/folders/gd/bylymgj907x1hy83j4td8ysm0000gn/T/ipykernel_35268/1532294146.py:35: UserWarning: Glyph 65341 (\N{FULLWIDTH RIGHT SQUARE BRACKET}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/var/folders/gd/bylymgj907x1hy83j4td8ysm0000gn/T/ipykernel_35268/1532294146.py:36: UserWarning: Glyph 65341 (\N{FULLWIDTH RIGHT SQUARE BRACKET}) missing from font(s) DejaVu Sans.
  plt.savefig('obb_quality_check.png', dpi=150, bbox_inches='tight')


✅ Saved obb_quality_check.png

📊 CROP QUALITY STATISTICS
Total crops: 956
Aspect ratios: 1.000 ± 0.000
✅ Excellent consistency - all crops are square


In [12]:
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd

class ImprovedOBBMandibleCropper:
    def __init__(self, model_path, margin_percent=0.1, use_clahe=False):
        self.model = YOLO(model_path)
        self.margin_percent = margin_percent
        self.use_clahe = use_clahe  # Make CLAHE optional
        
    def add_margin(self, points, margin_percent=0.1):
        """Add safety margin to OBB points"""
        center = points.mean(axis=0)
        expanded = center + (points - center) * (1 + margin_percent)
        return expanded
    
    def make_square_with_padding(self, image):
        """Add black padding to make image square"""
        h, w = image.shape[:2]
        
        if h == w:
            return image
        
        size = max(h, w)
        square = np.zeros((size, size), dtype=image.dtype)
        
        y_offset = (size - h) // 2
        x_offset = (size - w) // 2
        
        square[y_offset:y_offset+h, x_offset:x_offset+w] = image
        
        return square
    
    def detect_orientation(self, image):
        """
        Improved orientation detection using anatomical heuristics
        
        Strategy: The condyle (top) should have a rounded protrusion.
        We detect this by comparing edge density in top vs bottom quarters.
        """
        h, w = image.shape[:2]
        
        # Split into top and bottom quarters
        top_quarter = image[:h//4, :]
        bottom_quarter = image[-h//4:, :]
        
        # Apply edge detection
        edges_top = cv2.Canny(top_quarter, 50, 150)
        edges_bottom = cv2.Canny(bottom_quarter, 50, 150)
        
        # Count edge pixels (condyle has more curved edges)
        edge_density_top = np.sum(edges_top > 0) / edges_top.size
        edge_density_bottom = np.sum(edges_bottom > 0) / edges_bottom.size
        
        # If bottom has significantly more edges, likely upside down
        needs_flip = edge_density_bottom > edge_density_top * 1.3
        
        return needs_flip
    
    def normalize_without_clahe(self, image):
        """
        Gentle normalization that preserves intensity relationships
        
        Uses percentile-based clipping instead of CLAHE
        """
        # Clip extremes (removes outlier pixels from scatter/metal artifacts)
        p2, p98 = np.percentile(image, (2, 98))
        clipped = np.clip(image, p2, p98)
        
        # Normalize to 0-255 range
        normalized = ((clipped - clipped.min()) / (clipped.max() - clipped.min()) * 255).astype(np.uint8)
        
        return normalized
    
    def process_image(self, img_path, save_dir, save_raw=True):
        """
        Process image with improved orientation detection
        
        Args:
            save_raw: If True, save both raw and CLAHE versions for comparison
        """
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            return {'status': 'failed', 'reason': 'cannot_read'}
        
        results = self.model.predict(img_path, conf=0.5, verbose=False)
        
        metadata = {
            'filename': Path(img_path).name,
            'status': 'success',
            'detections': []
        }
        
        for result in results:
            if result.obb is None:
                metadata['status'] = 'no_detection'
                continue
                
            for obb in result.obb:
                points = obb.xyxyxyxy[0].cpu().numpy()
                cls = int(obb.cls[0])
                conf = float(obb.conf[0])
                class_name = self.model.names[cls]
                
                # Add margin
                points_expanded = self.add_margin(points, self.margin_percent)
                
                # Get OBB rectangle
                rect = cv2.minAreaRect(points_expanded.astype(np.float32))
                center, (width, height), angle = rect
                
                # Ensure width > height (horizontal orientation)
                if height > width:
                    width, height = height, width
                    angle += 90
                
                # Rotate entire image
                M_rotate = cv2.getRotationMatrix2D(center, angle, 1.0)
                img_h, img_w = img.shape[:2]
                rotated = cv2.warpAffine(img, M_rotate, (img_w, img_h), 
                                        flags=cv2.INTER_CUBIC,
                                        borderMode=cv2.BORDER_REPLICATE)
                
                # Get rotated center
                center_rotated = np.dot(M_rotate, [center[0], center[1], 1])
                
                # Crop aligned rectangle
                x = int(center_rotated[0] - width/2)
                y = int(center_rotated[1] - height/2)
                w = int(width)
                h = int(height)
                
                # Ensure crop is within bounds
                x = max(0, x)
                y = max(0, y)
                w = min(w, img_w - x)
                h = min(h, img_h - y)
                
                cropped = rotated[y:y+h, x:x+w]
                
                # ============================================
                # IMPROVED: Better orientation detection
                # ============================================
                needs_flip = self.detect_orientation(cropped)
                if needs_flip:
                    cropped = cv2.rotate(cropped, cv2.ROTATE_180)
                
                # Add padding to make square
                squared = self.make_square_with_padding(cropped)
                
                # Mirror right side
                if class_name == 'mandible_right':
                    squared = cv2.flip(squared, 1)
                
                # Resize
                resized = cv2.resize(squared, (224, 224), 
                                    interpolation=cv2.INTER_CUBIC)
                
                # ============================================
                # OPTIONAL: Save both raw and CLAHE versions
                # ============================================
                base_filename = f"{Path(img_path).stem}_{class_name}"
                
                # Save raw normalized version
                if save_raw:
                    normalized = self.normalize_without_clahe(resized)
                    output_raw = Path(save_dir) / f"{base_filename}_raw.png"
                    cv2.imwrite(str(output_raw), normalized)
                
                # Optionally save CLAHE version
                if self.use_clahe:
                    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
                    enhanced = clahe.apply(resized)
                    output_clahe = Path(save_dir) / f"{base_filename}_clahe.png"
                    cv2.imwrite(str(output_clahe), enhanced)
                else:
                    # Default: save only normalized version
                    normalized = self.normalize_without_clahe(resized)
                    output_path = Path(save_dir) / f"{base_filename}.png"
                    cv2.imwrite(str(output_path), normalized)
                
                metadata['detections'].append({
                    'class': class_name,
                    'confidence': conf,
                    'output_file': f"{base_filename}.png",
                    'orientation_flipped': needs_flip
                })
        
        return metadata

    def batch_process(self, input_dir, output_dir, save_raw=True):
        """Process all images"""
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        image_files = list(Path(input_dir).glob('*.jpg')) + \
                     list(Path(input_dir).glob('*.png'))
        
        print(f"🔍 Found {len(image_files)} images")
        
        results_log = []
        failed = []
        orientation_stats = {'flipped': 0, 'original': 0}
        
        for img_path in tqdm(image_files, desc="Processing OBB"):
            metadata = self.process_image(img_path, output_dir, save_raw=save_raw)
            results_log.append(metadata)
            
            # Track orientation statistics
            for det in metadata.get('detections', []):
                if det.get('orientation_flipped', False):
                    orientation_stats['flipped'] += 1
                else:
                    orientation_stats['original'] += 1
            
            if metadata['status'] != 'success':
                failed.append(metadata)
        
        success = sum(1 for r in results_log if r['status'] == 'success')
        total_crops = sum(len(r['detections']) for r in results_log)
        
        print("\n" + "="*60)
        print("📊 OBB PROCESSING SUMMARY")
        print("="*60)
        print(f"✅ Processed: {success}/{len(image_files)}")
        print(f"🖼️  Total crops: {total_crops}")
        print(f"❌ Failed: {len(failed)}")
        print(f"\n🔄 Orientation Stats:")
        print(f"   Original: {orientation_stats['original']}")
        print(f"   Flipped: {orientation_stats['flipped']} ({orientation_stats['flipped']/total_crops*100:.1f}%)")
        
        if failed:
            print("\n⚠️  Failed images:")
            for f in failed[:10]:
                print(f"   - {f['filename']}")
        
        # Save log
        pd.DataFrame(results_log).to_csv(
            Path(output_dir) / 'obb_processing_log.csv', index=False
        )
        
        return results_log

if __name__ == "__main__":
    # Configuration
    MODEL_PATH = "mandible_yolo_obb/exp_v1_n52/weights/best.pt"
    INPUT_DIR = "/Users/rieno/Downloads/DATASET MENTAH/MATCHED_DATASET_3/uncropped"
    OUTPUT_DIR = "cropped_obb_hemijaws_v2"
    
    # Process WITHOUT CLAHE (recommended for age estimation)
    cropper = ImprovedOBBMandibleCropper(
        MODEL_PATH, 
        margin_percent=0.10,
        use_clahe=False  # Disable CLAHE
    )
    
    # Save both versions for comparison
    results = cropper.batch_process(INPUT_DIR, OUTPUT_DIR, save_raw=True)
    
    print(f"\n✅ Done! Check: {OUTPUT_DIR}/")

🔍 Found 483 images


Processing OBB: 100%|██████████| 483/483 [00:30<00:00, 16.09it/s]


📊 OBB PROCESSING SUMMARY
✅ Processed: 483/483
🖼️  Total crops: 967
❌ Failed: 0

🔄 Orientation Stats:
   Original: 951
   Flipped: 16 (1.7%)

✅ Done! Check: cropped_obb_hemijaws_v2/
